In [47]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna

In [69]:
df_train = pd.read_csv("data/final/final_training.csv")
df_train = df_train[df_train['pot_doc'] != '[]']
X_train_df=pd.DataFrame(data = np.array(df), columns = df.columns).drop(columns="documentid").drop(columns="Unnamed: 0")
y_train_df = pd.DataFrame(data = np.array(df['documentid']), columns = ['documentid'])
X_train_df

df_train = pd.read_csv("data/final/final_test.csv")
X_test_df=pd.DataFrame(data = np.array(df), columns = df.columns).drop(columns="documentid").drop(columns="Unnamed: 0")
y_test_df = pd.DataFrame(data = np.array(df['documentid']), columns = ['documentid'])


In [86]:
def compute_accuracy(x, y, z,a):
    results = []
    for row in X_train_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2)

            #+ (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return np.sum(results == np.array(y_train_df['documentid']))/len(X_train_df)
    
def compute_accuracy2(x, y, z, a):
    results = []
    for row in X_test_df.iterrows():
        if (len(eval(row[1]['pot_doc']))==0):
            continue
        dists = []
        
        for idx in range(len(eval(row[1]['pot_doc']))):
            dists.append((x*(100-eval(row[1]['pot_price'])[idx])/100)**2 
            + (y*(1-eval(row[1]['pot_date'])[idx]))**2
            + (z*(100-eval(row[1]['pot_address'])[idx])/100)**2
            #+ (a*eval(row[1]['pot_name'])[idx])**2
            )
        dists = np.array(dists)
        results.append(eval(row[1]['pot_doc'])[dists.argmin()])
    results = np.array(results)
    return results

In [90]:

# dfs = pd.DataFrame()
# ss = pd.DataFrame()

# dfs['result'] = ss.apply(lambda row: (row['documentid'] in row['pot_doc']), axis=1)
# print(dfs)
# num_trues = dfs['result'].value_counts()[True]
# print(num_trues)
def objective(trial):
    x = trial.suggest_float('x', 0, 10)
    y = trial.suggest_float('y', 0, 10)
    z = trial.suggest_float('z', 0, 10)
    #a = trial.suggest_float('a', 0, 10)
    return 1-compute_accuracy(x,y,z,0)

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}



[I 2023-01-29 07:19:52,189] A new study created in memory with name: no-name-5dc8158b-1891-4484-9432-80222161bacd
[I 2023-01-29 07:19:52,394] Trial 0 finished with value: 0.16801619433198378 and parameters: {'x': 6.14440991083287, 'y': 1.8352769801072444, 'z': 4.032436107549383}. Best is trial 0 with value: 0.16801619433198378.
[I 2023-01-29 07:19:52,599] Trial 1 finished with value: 0.16801619433198378 and parameters: {'x': 5.304130593339887, 'y': 9.33978069930365, 'z': 0.0073807567742734115}. Best is trial 0 with value: 0.16801619433198378.
[I 2023-01-29 07:19:52,799] Trial 2 finished with value: 0.16801619433198378 and parameters: {'x': 0.8255639566128614, 'y': 6.351277713350012, 'z': 2.0891393563877445}. Best is trial 0 with value: 0.16801619433198378.
[I 2023-01-29 07:19:52,995] Trial 3 finished with value: 0.16801619433198378 and parameters: {'x': 2.1254821851610295, 'y': 9.12813858870134, 'z': 0.03315975666312254}. Best is trial 0 with value: 0.16801619433198378.
[I 2023-01-29 0

{'x': 6.14440991083287, 'y': 1.8352769801072444, 'z': 4.032436107549383}

In [91]:
res = compute_accuracy2(study.best_params['x'], 1, study.best_params['y'], 0)

real = list(y_test_df['documentid'])

print(len(res), len(real))

494 494


In [92]:
def calculate_percentage(list_a, list_b):
    common_elements = set(list_a) & set(list_b)
    percentage = (len(common_elements) / len(list_a)) * 100
    return percentage
print(calculate_percentage(res,real))

84.0080971659919
